In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# Workflow Control
This notebook provides an end-to-end procedure for implementing the method described in "Unsupervised Deep Clustering of Seismic Data: Monitoring the Ross Ice Shelf, Antarctica."

<a id="contents"></a>
***
## Table of Contents

1. [Initialize Project Environment](#section1)
2. [Seismic Pre-Processing](#section2)
3. [Set Universal Experiment Parameters](#section3)
4. [Pre-train DEC Model](#section4)
4. [Train DEC Model](#section5)
5. [Cluster Entire Dataset](#section6)

Appendices  
A. [Evaluate Optimal Number of Clusters](#appendixA)

In [ ]:
import datetime
import os
import sys

from IPython.display import Markdown as md
import numpy as np
from torch.utils.data import DataLoader, Subset
from torchsummary import summary
from torchvision import transforms

from RISCluster import models, plotting, utils
from RISCluster.networks import AEC, DEC
from RISProcess.io import config

<a id="section1"></a>
***
## 1 Initialize Project Environment
The default project structure is:<br>
`/Project Folder
├── Config
├── Data
│   ├── Meteo
│   ├── Ice
│   ├── Seismo
│   │   ├── MSEED
│   │   └── StationXML
└── Outputs
`
<br>Note that the raw seismic data from 1-Dec-2014 to 1-Dec-2016 is nearly 1 TB. It may be practical to split out the project's `Data` folder onto a disk with more storage.  If so, set the path to the data storage below.

In [ ]:
# Main project folder to save outputs:
project_folder = '.'
# Path to configuration files:
path_config = f"{project_folder}/Config"
# Path to folder containing data, including HDF file for ML workflow:
path_data = f"{project_folder}/Data"
# Path to raw seismic data:
path_data_seismo = f"{path_data}/Seismo"
# Path to save workflow outputs (ML models, figures, results, etc.)
path_output = f"{project_folder}/Outputs"
# Path to HDF dataset:
fname_dataset = f"{path_data}/RISData.h5"
# Path to save paper-ready figures:
figure_savepath = f"{path_output}/Figures"

utils.init_project_env([path_config, path_data, path_data_seismo, path_output, figure_savepath])

<a href="#contents">Return to Top</a>
<a id="section2"></a>
***
## 2 Seismic Pre-Processing
Four workflows are provided for obtaining and pre-processing seismic data.  The recommended workflow makes use of sections 2.1, 2.3, and 2.4; section 2.2 is not required for the rest of the workflow, but instead provides a pipeline to save pre-processed data to disk.

### 2.1 Download Data
In this workflow, seismic data is downloaded using the FDSN mass data downloader. Data set parameters are stored in the configuration file to configpath; MSEED data are saved to `datapath/MSEED`; and station XML data are saved to `datapath/StationXML`.  The MSEED data are saved according to the following convention:
`Network.Station..Channel__YYYYMMDDTHHMMSSZ__YYYYMMDDTHHMMSSZ.mseed`

In [ ]:
parameters = {
    'start': '20141201T0000',
    'stop': '20141203T0000',
    'mode': 'download',
    'datapath': path_data_seismo,
    'network': 'XH',
    'station': '*',
    'channel': 'HH*',
}
config_file = config('w', path=path_config, parameters=parameters)
print("Run the following in Terminal:")
md(f"`dlfdsn {config_file}`")

### 2.2 Pre-process Data
In this workflow, raw seismic data is read from `datapath`, processed, and saved to `writepath` according to the following file naming conventions:<br>
`MSEED/Network/Station/Network.Station.Channel.YYYY.DAY.mseed`

For the input data, two file formats are available.
<br>**Format 1:**
<br>`Network.Station.Channel.YYYY.DAY.mseed`
<br>**Format 2:**
<br>`Network.Station..Channel__YYYYMMDDTHHMMSSZ__YYYYMMDDTHHMMSSZ.mseed`

In [ ]:
parameters = {
    'start': '20141201T0000',
    'stop': '20141203T0000',
    'mode': 'preprocess',
    'sourcepath': path_data_seismo,
    'name_format': 2,
    'writepath': f"{path_data_seismo}/Preprocessed",
    'parampath': f"{path_data_seismo}/Preprocessed",
    'network': 'XH',
    'channel': 'HHZ',
    'taper': 60,
    'prefeed': 60,
    'fs2': 50,
    'cutoff': '3, 20',
    'output': 'acc',
    'prefilt': '0.004, 0.01, 500, 1000',
    'waterlevel': 14,
    'detector': 'z',
    'on': 8,
    'off': 4,
    'num_workers': 4
}
config_file = config('w', path=path_config, parameters=parameters)
print("Run the following in Terminal:")
md(f"`process {config_file}`")

### 2.3 Detect Events & Build Catalogue
In this workflow, raw seismic data in `datapath` are processed in 24-hour segments, and an event detection algorithm is applied. The results of the event detector are compiled into a catalogue that is saved to disk at `writepath`. This catalogue serves as a useful pointer for follow-on processing of events of interest, rather than continuous data.

#### 2.3.1 Build Unsorted Catalogue

In [ ]:
parameters = {
    'start': '20141201T0000',
    'stop': '20141203T0000',
    'mode': 'detect',
    'sourcepath': path_data_seismo,
    'name_format': 2,
    'writepath': path_data,
    'parampath': path_data,
    'network': 'XH',
    'channel': 'HHZ',
    'taper': 60,
    'prefeed': 60,
    'fs2': 50,
    'cutoff': '3, 20',
    'output': 'acc',
    'prefilt': '0.004, 0.01, 500, 1000',
    'waterlevel': 14,
    'detector': 'z',
    'on': 8,
    'off': 4,
    'num_workers': 4
}
config_file = config('w', path=path_config, parameters=parameters)
print("Run the following in Terminal:")
md(f"`process {config_file}`")

#### 2.3.2 Clean Catalogue
Remove duplicate detections, and if desired, detections that occur within a window (s) following an initial detection.

In [ ]:
window = 10
!cleancat {path_data + '/catalogue.csv'} --dest {path_data + '/catalogue2.csv'} --window $window

### 2.4 Build HDF Database from Catalogue
In this workflow, a catalogue of detections at catalogue is used to process raw seismic data in `datapath`. In addition to pre-processing, the traces, spectrograms, and metadata of the detections are saved to an HDF database located at `writepath`. Because this workflow is implemented in parallel and results are returned asynchronously, a new catalogue is saved to `writepath.csv` that corresponds to the indexing within the HDF dataset. The index within `writepath.csv` corresponds to the original catalogue at catalogue.

In [ ]:
parameters = {
    'start': '20141201T0000',
    'stop': '20141203T0000',
    'mode': 'cat2h5',
    'sourcepath': path_data_seismo,
    'name_format': 2,
    'writepath': fname_dataset,
    'catalogue': f"{path_data}/catalogue2.csv",
    'parampath': path_data,
    'network': 'XH',
    'channel': 'HHZ',
    'taper': 10,
    'prefeed': 10,
    'fs2': 50,
    'cutoff': '3, 20',
    'T_seg': 4,
    'NFFT': 256,
    'tpersnap': 0.4,
    'overlap': 0.9,
    'output': 'acc',
    'prefilt': '0.004, 0.01, 500, 1000',
    'waterlevel': 14,
    'detector': 'z',
    'on': 8,
    'off': 4,
    'det_window': 5,
    'num_workers': 2
}
config_file = config('w', path=path_config, parameters=parameters)
print("Run the following in Terminal:")
md(f"`process {config_file}`")

<a href="#contents">Return to Top</a>
<a id="section3"></a>
***
## 3 Set Universal Parameters

In [ ]:
exp_name = "FullArray"

# Get the number of samples in the dataset.
!query_H5size $fname_dataset

# Image Sample Indexes for Example Waveforms:
img_index = [403049, 334383, 300610, 381290]

# Generate new sample index for data set?
genflag = False

In [ ]:
if genflag:
    M = 125000
    !GenerateSampleIndex $M $fname_dataset $path_data

In [ ]:
universal = {
    'exp_name': exp_name,
    'fname_dataset': fname_dataset,
    'savepath': f'./Outputs',
    'indexpath': './Data/TraValIndex_M=125000.pkl',
    'configpath': './Config'
}
device = utils.set_device(0)
transform = 'sample_norm_cent'

<a href="#contents">Return to Top</a>
<a id="section4"></a>
***
## 4 Pre-train DEC Model

### 4.1 Autoencoder Architecture

In [ ]:
model_display = AEC().to(device)
summary(model_display, (1, 87, 100))

### 4.2 Configure Pre-training

In [ ]:
parameters = {
    'mode': 'pretrain',
    'n_epochs': 500,
    'show': False,
    'send_message': True,
    'early_stopping': True,
    'patience': 10,
    'transform': 'sample_norm_cent',
    'img_index': str(img_index)[1:-1],
    'km_metrics': False,
    'klist': '2, 20',
    'tb': True,
    'tbport': 6999,
    'workers': 16
}
hyperparameters = {
    'batch_size': '512, 1024, 2048',
    'lr': '0.001, 0.01, 0.1'
}
init_path = utils.config_training(universal, parameters, hyperparameters)

### 4.3 View Detection Examples

In [ ]:
fig = plotting.view_detections(fname_dataset, img_index)

In [ ]:
fig.savefig(f"{figure_savepath}/DetectionExamples.eps", dpi=300, facecolor='w')

### 4.4 Train Autoencoder

In [ ]:
print("Run the following in Terminal:")
md(f"`runDEC {init_path}`")

<a id="BestAEC"></a>
### 4.5 Select Best Autoencoder Run
Use Tensorboard to view outputs from the various hyperparameter runs.

In [ ]:
batch_size = 256
LR = 0.001

expserial = 'Exp20201220T072755'
runserial = f'Run_BatchSz={batch_size}_LR={LR}'
exp_path = f"{path_output}/Models/AEC/{expserial}/{runserial}"

AEC_weights = f"{exp_path}/AEC_Params_Final.pt"

Return to [Section 5.2](#ConfigDCM)<br>
Return to [Section 7](#section5)

### 4.6 Evaluate Autoencoder Performance

#### 4.6.1 Load Data and Model Parameters

In [ ]:
dataset = utils.H5SeismicDataset(
    fname_dataset,
    transform=transforms.Compose(
        [utils.SpecgramShaper(), utils.SpecgramToTensor()]
    )
)
display_subset = Subset(dataset, img_index)

model = AEC().to(device)
model = utils.load_weights(model, AEC_weights, device)

#### 4.6.2 Training and Validation History

In [ ]:
fig = plotting.view_history_AEC(f"{exp_path}/AEC_history.csv", show=True)

#### 4.6.3 Input, Latent Space, and Reconstruction

In [ ]:
fig = plotting.compare_images(
    model,
    0,
    [img_index[0]],
    fname_dataset,
    device
)

In [ ]:
fig.savefig(f"{figure_savepath}/CompareInOut.eps", dpi=300, facecolor='w')

[Return to Top](#contents)
<a id="section5"></a>
***
## 5. Train DEC Model

### 5.1 DEC Model Architecture

In [ ]:
model_display = DEC(n_clusters=5).to(device)
summary(model_display, (1, 87, 100))

<a id="ConfigDCM"></a>
### 5.2 Configure Training
Run [4.5](#BestAEC) first to get AEC weights.

In [ ]:
parameters = {
    'mode': 'train',
    'n_epochs': 400,
    'update_interval': -1,
    'show': False,
    'send_message': True,
    'saved_weights': AEC_weights,
    'transform': 'sample_norm_cent',
    'tb': True,
    'tbport': 6999,
    'workers': 16,
    'init': 'gmm'
}
hyperparameters = {
    'n_clusters': '8',
    'batch_size': '1024',
    'lr': '0.001',
    'gamma': '0.1',
    'tol': 0.002
}
init_path = utils.config_training(universal, parameters, hyperparameters)

### 5.3 Train DEC Model

Run the following in Terminal:

In [ ]:
md(f"`runDEC {init_path}`")

To specify which CUDA device(s) is(are) used, prepend the following:

In [ ]:
md(f"`CUDA_VISIBLE_DEVICES=#`")

<a id="BestDEC"></a>
### 5.4 Select Best DEC Run
Use Tensorboard to view outputs from the various hyperparameter runs.

In [ ]:
n_clusters = 8
batch_size = 1024
LR = 0.001

expserial = 'Exp20201226T120534'
runserial = f'Run_Clusters={n_clusters}_BatchSz={batch_size}_LR={LR}_gamma=0.05_tol=0.002'
exp_path = f"{path_output}/Models/DEC/{expserial}/{runserial}"
DEC_weights = f"{exp_path}/DEC_Params_Final.pt"

Return to [Section 6](#section6)

### 5.5 Evaluate DEC Training

#### 5.5.1 Load Data and Model Parameters

In this step, two models are instantiated: one before clustering, and one after clustering.

In [ ]:
dataset = utils.H5SeismicDataset(
    fname_dataset,
    transform=transforms.Compose(
        [utils.SpecgramShaper(), utils.SpecgramToTensor()]
    )
)
index_tra, _ = utils.load_TraVal_index(fname_dataset, universal['indexpath'])
tra_dataset = Subset(dataset, index_tra)
dataloader = DataLoader(tra_dataset, batch_size=1024, num_workers=16)

DEC_weights1 = f"{exp_path}/DEC_Params_Initial.pt"
DEC_weights2 = DEC_weights

model1 = DEC(n_clusters).to(device)
model1 = utils.load_weights(model1, DEC_weights1, device)
model2 = DEC(n_clusters).to(device)
model2 = utils.load_weights(model2, DEC_weights2, device)

centroids1 = model1.clustering.weights.detach().cpu().numpy()
centroids2 = model2.clustering.weights.detach().cpu().numpy()

_, labels1, data1 = models.infer(dataloader, model1, device, v=True)
_, labels2, data2 = models.infer(dataloader, model2, device, v=True)

#### 5.5.2 Training History

In [ ]:
fig = view_history_DEC([f"{exp_path}/DEC_history.csv", f"{exp_path}/Delta_history.csv"], show=True)

#### 5.5.3 Clustering Results

In [ ]:
p = 2
fig = plotting.cluster_gallery(
    model2,
    dataloader.dataset,
    fname_dataset,
    index_tra,
    device,
    data2,
    labels2,
    centroids2,
    p,
    True,
    True
)

In [ ]:
fig.savefig(f"{figure_savepath}/gallery.eps", dpi=300, facecolor='w')

#### 5.5.4 t-SNE Results

In [ ]:
if sys.platform == 'darwin':
    from sklearn.manifold import TSNE
elif sys.platform == 'linux':
    from cuml import TSNE

M = len(data1)
results1 = TSNE(n_components=2, perplexity=int(M/50), early_exaggeration=2000, learning_rate=int(M/25), n_iter=3000, verbose=0, random_state=2009).fit_transform(data1.astype('float64'))
results2 = TSNE(n_components=2, perplexity=int(M/50), early_exaggeration=2000, learning_rate=int(M/25), n_iter=3000, verbose=0, random_state=2009).fit_transform(data2.astype('float64'))
fig1 = plotting.view_TSNE(results1, labels1, 't-SNE Results: GMM', True)
fig2 = plotting.view_TSNE(results2, labels2, 't-SNE Results: DEC', True)

In [ ]:
fig1 = view_TSNE(results1, labels1, 't-SNE Results: GMM', show=True)
fig2 = view_TSNE(results2, labels2, 't-SNE Results: DEC', show=True)

In [ ]:
fig1.savefig(f"{figure_savepath}/tSNE_i.pdf", dpi=300, facecolor='w')
fig2.savefig(f"{figure_savepath}/tSNE_f.pdf", dpi=300, facecolor='w')

#### 5.5.5 DEC Dashboard

In [ ]:
p = 2
fig = plotting.centroid_dashboard(
    data2,
    labels2,
    centroids2,
    n_clusters,
    p,
    True
)

#### 5.5.6 View Centroid Distance Matrix

In [ ]:
p = 2
fig = plotting.centroid_distances(
    data2,
    labels2,
    centroids2,
    n_clusters,
    p,
    True
)

#### 5.5.7 View Latent Space

In [ ]:
p = 2
fig = plotting.view_latent_space(
    data1,
    data2,
    labels1,
    labels2,
    centroids1,
    centroids2,
    n_clusters,
    p,
    True,
    True
)

In [ ]:
fig.savefig(f"{figure_savepath}/zspace.pdf", dpi=300, facecolor='w')

#### 5.5.8 Cluster CDFs

In [ ]:
p = 2
fig = plotting.view_class_cdf(
    data1,
    data2,
    labels1,
    labels2,
    centroids1,
    centroids2,
    n_clusters,
    p,
    True,
    True
)

In [ ]:
fig.savefig(f"{figure_savepath}/CDF.pdf", dpi=300, facecolor='w')

#### 5.5.9 Cluster PDFs

In [ ]:
p = 2
fig = plotting.view_class_pdf(
    data1,
    data2,
    labels1,
    labels2,
    centroids1,
    centroids2,
    n_clusters,
    p,
    True,
    True
)

In [ ]:
fig.savefig(f"{figure_savepath}/PDF.pdf", dpi=300, facecolor='w')

[Return to Top](#contents)
<a id="section6"></a>
***
## 6 Cluster Entire Dataset

### 6.1 Configure Inference
Run [Section 5.4](#BestDEC) first to get DEC weights.

In [ ]:
parameters = {
    'mode': 'predict',
    'send_message': False,
    'saved_weights': DEC_weights,
    'transform': 'sample_norm_cent',
    'workers': 16,
    'tb': False
}
init_path = utils.config_training(universal, parameters)

### 6.2 Run DEC Model in Inference

In [ ]:
print("Run the following in Terminal:")
md(f"`runDEC {init_path}`")

### 6.3 Calculate Dataset Statistics
Set path to clustering results and data set:

In [ ]:
path_to_catalogue = f"{fname_dataset}.csv"
path_to_labels = f"{exp_path}/Labels.csv"
catalogue = utils.LabelCatalogue([path_to_catalogue, path_to_labels])

#### 6.3.1 Station Statistics
View occurrence frequencies by station and label.

In [ ]:
catalogue.station_statistics().sort_values(by="N", ascending=False)

#### 6.3.2 Amplitude Statistics
View amplitude characteristics for each class.

In [ ]:
catalogue.amplitude_statistics()

#### 6.3.3 Seasonal Statistics
Compare occurrence frequencies in austral winter (JFM) to austral summer (JJA).

In [ ]:
catalogue.seasonal_statistics(mode=True)

#### 6.3.4 Peak Frequency Statistics
View average peak frequencies for each class:

In [ ]:
catalogue.get_peak_freq(fname_dataset, batch_size=2048, workers=12)

### 6.4 View Environmental Data & Detection Statistics

#### 6.4.1 View Station DR02

In [ ]:
station = "DR02"
aws = "gil"
fig = plotting.view_series(
    station,
    aws,
    path_data,
    path_to_catalogue,
    path_to_labels,
    env_vars=["sea_ice_conc","temp","wind_spd"],
    freq="hour",
    maxcounts=20,
    title=f"Station {station} Inter-annual Scale",
    show=True
)

In [ ]:
fig.savefig(f"{figure_savepath}/{station}.eps", dpi=300, facecolor='w')

#### 6.4.2 View Station RS09

In [ ]:
station = "RS09"
aws = "mgt"
start = datetime.datetime(2016,6,15)
stop = datetime.datetime(2016,7,15)
fig1 = plotting.view_series(
    station,
    aws,
    path_data,
    path_to_catalogue,
    path_to_labels,
    env_vars=["temp","wind_spd","tide"],
    vlines=[start, stop],
    freq="hour",
    maxcounts=30,
    figsize=(12,9),
    title=f"Station {station} Interannual Scale",
    show=True
)
fig2 = plotting.view_series(
    station,
    aws,
    path_data,
    path_to_catalogue,
    path_to_labels,
    env_vars=["temp","wind_spd","tide"],
    times=[start, stop],
    freq="hour",
    maxcounts=20,
    figsize=(6,9),
    title=f"Station {station} Weekly Scale",
    showlabels=False,
    show=True
)

In [ ]:
fig1.savefig(f"{figure_savepath}/{station}_ia.eps", dpi=300, facecolor='w')
fig2.savefig(f"{figure_savepath}/{station}_wk.eps", dpi=300, facecolor='w')

<a href="#contents">Return to Top</a>
<a id="appendixA"></a>
***
## Appendix A: Test for Optimal Number of Clusters

### A.1 Load Data
Run <a href="#BestAEC">4.5</a> first to get AEC weights.

In [ ]:
index_tra, _ = utils.load_TraVal_index(fname_dataset, universal['indexpath'])

tra_dataset = Subset(dataset, index_tra)
dataloader = DataLoader(tra_dataset, batch_size=512, num_workers=16)

model = AEC().to(device)
model = utils.load_weights(model, AEC_weights, device)

### A.2 Compute K-means Metrics

In [ ]:
klist = '2, 20'
klist = np.arange(int(klist.split(',')[0]), int(klist.split(',')[1])+1)
inertia, silh, gap_g, gap_u = models.kmeans_metrics(dataloader, model, device, klist)

### A.3 Plot Metrics

In [ ]:
fig = plotting.view_cluster_stats(klist, inertia, silh, gap_g, gap_u, show=True)
np.save('kmeans_inertia', inertia)